In [2]:
import requests
import pandas as pd

from global_utilities import get_secret

base_url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsedates/v1.0/"

headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': get_secret("RAPIDAPI_KEY")
}

In [21]:
from flights.rapidapi import parse_data

from datetime import date, timedelta
from tqdm import tqdm_notebook as tqdm
from time import sleep

In [69]:
dfs = []

In [70]:
def query_flights(origin, destination, day):
    
    for attemp_num in range(10):
        response = requests.get(f"{base_url}ES/EUR/en-US/BCN/CAG/{day:%Y-%m-%d}", headers=headers)
        
        if response.status_code == 200:
            return response
    
        elif response.status_code == 429:
            sleep(0.5 * attemp_num)
            
        else:
            response.raise_for_status()

In [71]:
for x in tqdm(range(360)):
    day = date.today() + timedelta(x)

    response = query_flights("BCN", "CAG", day)
    
    data = response.json()
    
    if data["Quotes"]:
        dfs.append(parse_data(data))

In [72]:
df = pd.concat(dfs)

In [73]:
df

,MinPrice,Direct,QuoteDateTime,DepartureDate,Carrier,Origin,Destination
,,,,,,,
1,96.0,False,2020-01-11 18:13:00,2020-01-13,Ryanair,BCN,CAG
1,70.0,False,2020-01-11 18:14:00,2020-01-14,Ryanair,BCN,CAG
1,78.0,False,2020-01-11 18:16:00,2020-01-15,Vueling Airlines,BCN,CAG
1,65.0,False,2020-01-11 18:09:00,2020-01-16,Ryanair,BCN,CAG
1,97.0,False,2020-01-11 15:44:00,2020-01-17,Ryanair,BCN,CAG
...,...,...,...,...,...,...,...
1,83.0,False,2020-01-04 17:40:00,2020-10-19,Iberia,BCN,CAG
1,44.0,True,2020-01-08 23:03:00,2020-10-21,Vueling Airlines,BCN,CAG
1,73.0,False,2020-01-05 12:52:00,2020-10-22,Ryanair,BCN,CAG
